# Testing Lindblad-Poisson error model

From our notes, we obtained that a Lindblad error model with generalized Paulis as jump operators induces the following probability distribution of Pauli errors on qudits. 

$$
P(k) = \frac{1}{d} \sum_{l=0}^{d - 1} e^{-\alpha \sin^2 (l \pi /d)} e^{2 \pi i l k /d},
$$

with $\alpha = 4 \gamma t$. 

In [1]:
import sympy
import numpy as np
import jax.numpy as jnp
import jax
from coherentinfo.errormodel import ErrorModelLindblad
# If you disable the following you will notice that the results
# are accurate only up to 10^{-7} or similar
jax.config.update("jax_enable_x64", True)

First let us check that it is correctly normalized

In [2]:
def prob(error: int, gamma: float, d: int) -> float:
    @jax.jit()
    def func(l):
        return jnp.exp(-4 * gamma * jnp.sin(l * jnp.pi / d)**2) * jnp.exp(2 * jnp.pi * 1j * l * error / d) / d
    l_vec = jnp.arange(0, d)
    result = jnp.sum(jax.vmap(func)(l_vec))
    return jnp.real(result)

In [3]:
d = 38
gamma = 0.01
error_vec = jnp.arange(0, d)
norm_factor = jnp.sum(
    jax.vmap(prob, in_axes=(0, None, None))(error_vec, gamma, d))
print("Normalization factor: {}".format(norm_factor))

Normalization factor: 1.0000000000000102


as expected! 

Furthermore, we can check that the probabilities are symmetric

In [ ]:
error = 5
print(prob(error, gamma, d))
print(prob(d - error, gamma, d))

8.168916931783343e-13
8.170304710564125e-13


We can also check that these are the probabilities generated by the corresponding error model

In [ ]:
em_lindblad = ErrorModelLindblad(num_subsys=1, d=d, gamma_t=gamma)
print(em_lindblad.probs)

Array([ 9.80296696e-01,  9.80247684e-03,  4.90115674e-05,  1.63370530e-07,
        4.08424482e-10,  8.16558626e-13,  1.09461051e-15,  1.07552856e-16,
       -7.19910243e-17,  3.53883589e-16,  4.08527379e-16,  5.30825384e-16,
       -3.29597460e-16, -1.80411242e-16, -4.51028104e-17, -4.82253126e-16,
       -7.28583860e-16, -1.59594560e-15,  5.30825384e-16,  3.46944695e-18,
       -6.17561557e-16, -5.68989300e-16,  5.55111512e-16,  1.21430643e-15,
       -4.26741975e-16, -7.70217223e-16,  1.05297715e-15,  1.24206201e-15,
       -1.00527225e-15, -2.15105711e-16, -2.97505076e-16,  2.77555756e-17,
        2.60035049e-15,  8.17103329e-13,  4.08422955e-10,  1.63370531e-07,
        4.90115674e-05,  9.80247684e-03], dtype=float64)